## Проект по SQL

### Описание проекта

Вам передали базу данных одного из конкурирующих сервисов - приложения для тех, кто любит читать. В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги. Эти данные помогут сформулировать ценностное предложение для нового продукта.

#### Таблица books

Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

#### Таблица authors

Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

#### Таблица publishers

Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

#### Таблица ratings

Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

#### Таблица reviews

Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

### План работы

- Посчитать количество книг, выпущенных после 1 января 2000 года;
- Посчитать количество пользовательских обзоров и среднюю оценку для каждой книги;
- Определить издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры);
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;
- Посчитать среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#pd.io.sql.read_sql(query, con = engine)

### Шаг 1. Посмотрим на таблицы базы

In [3]:
query_books = ''' SELECT *
        FROM books
        LIMIT 5
    '''

In [4]:
books = pd.io.sql.read_sql(query_books, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query_authors = ''' SELECT *
        FROM authors
        LIMIT 5
    '''
authors = pd.io.sql.read_sql(query_authors, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query_ratings = ''' SELECT *
        FROM ratings
        LIMIT 5
    '''
ratings = pd.io.sql.read_sql(query_ratings, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query_reviews = ''' SELECT *
        FROM reviews
        LIMIT 5
    '''
reviews = pd.io.sql.read_sql(query_reviews, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
query_publishers = ''' SELECT *
        FROM publishers
        LIMIT 5
    '''
publishers = pd.io.sql.read_sql(query_publishers, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Данные выглядят аккуратно, все типы соответствуют содержанию столбцов, заголовки столбцов исчерпывающе отражают суть данных.

### Шаг 2. Ответы на вопросы

Для подсчета кол-ва книг, выпущенных после 1 января 2000 года, достаточно обратиться к таблице books

In [9]:
query_books_count = ''' SELECT 
        COUNT (book_id) AS cnt
        FROM books
        WHERE publication_date > '2000-01-01'
    '''
books_count = pd.io.sql.read_sql(query_books_count, con = engine)
books_count

,cnt
0,819


В базе есть сведения о 819 книгах, вышедших после 1 января 2000 года.

In [10]:
query_ratings_reviews = '''SELECT
        ratings.book_id AS book_id,
        COUNT(reviews.text) AS total_reviews,
        AVG(ratings.rating) AS mean_rating
        FROM
            ratings
        LEFT JOIN reviews ON 
            (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
        GROUP BY 
            ratings.book_id
        ORDER BY 
            ratings.book_id
'''
ratings_reviews = pd.io.sql.read_sql(query_ratings_reviews, con = engine)
ratings_reviews

,book_id,total_reviews,mean_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


На этом шаге получена таблица, в которой каждому id книги соответствует ее средний рейтинг и число обзоров.

In [11]:
query_publishers_books = '''SELECT
        publishers.publisher AS publisher,
        COUNT(books.book_id) AS total_books
        FROM
            books
        LEFT JOIN publishers ON 
            publishers.publisher_id = books.publisher_id
        GROUP BY 
            publishers.publisher
        HAVING
            SUM(books.num_pages) > 50
        ORDER BY 
            total_books DESC
        LIMIT 5
'''
publishers_books = pd.io.sql.read_sql(query_publishers_books, con = engine)
publishers_books

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Больше всего книг толще 50 страниц - на счету издательства Penguin Books: 42 книги.

In [15]:
query_author_rate = '''SELECT
    authors.author,
    AVG(filtered.average_rating) AS avg_rating
    FROM
        (SELECT
            book_id AS book_id,
            COUNT(rating_id) AS rating_counts,
            AVG(rating) AS average_rating
        FROM
            ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50) 
        AS filtered
    INNER JOIN books ON books.book_id = filtered.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY authors.author
    ORDER BY avg_rating DESC
    LIMIT 1
'''
author_rate = pd.io.sql.read_sql(query_author_rate, con = engine)
author_rate

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Самым рейтинговым автором в базе является Джоан Роулинг.

In [ ]:
query_mean_reviews = '''
        SELECT
        AVG(sub.count)
        FROM
        (SELECT COUNT(*)
            FROM reviews
            WHERE username IN
                (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50)
            GROUP BY username)
            AS sub;
        '''

mean_reviews = pd.io.sql.read_sql(query_mean_reviews, con = engine)
mean_reviews

Пользователи, поставившие более 50 оценок, в среднем оставляют 24 отзыва.

### Выводы

- В базе есть сведения о 819 книгах, вышедших после 1 января 2000 года. В среднем это 40 книг в год. Это количество не выглядит  большим - хотя это смотря с чем сравнивать.
- Больше всего книг толще 50 страниц - на счету издательства Penguin Books: 42 книги. Это достаточно серьезное издательство с почти вековой историей, так что их лидерство в этом рейтинге неудивительно.
- Самым рейтинговым автором в базе является Джоан Роулинг, автор "Поттерианы"
- Пользователи, поставившие более 50 оценок, в среднем оставляют 24 отзыва. Это может говорить о заинтересованности читателей в интерактивности процесса чтения. Как и любое коллекционирование, коллекционирование книг и их сортировка в цифровом мире дает возможности для новых видов взаимодействия: отзывы, рейтинги, обмен мнениями, списки имеющихся, прочитанных и запланированных к прочтению книг и так далее.